In [ ]:
from ib_insync import *
import requests
import pandas as pd
import schedule
import time
import datetime;
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
# https://stackoverflow.com/questions/44933518/how-to-remove-runtimewarning-errors-from-code

util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)
# https://algotrading101.com/learn/ib_insync-interactive-brokers-api-guide/
# https://www.reddit.com/r/interactivebrokers/comments/1cccg0t/ib_insync_reqhistoricaldata_with_keepuptodatetrue/
# https://github.com/erdewit/ib_insync/issues/653

In [ ]:
#WORKING
counter_of_numbers = 0
ib = IB()
ib.connect(host='127.0.0.1', port=7496, clientId=0)
# https://www.elitetrader.com/et/threads/cant-connect-to-api-interactive-brokers.361205/
# needed to change socket port to 7496 and uncheck the Read-Only API box

In [ ]:
#tester = ib.trades()
#tester1 = tester[-1]
#tester2 = tester1.contract.symbol, tester1.order.lmtPrice
#tester

In [ ]:
#posits = ib.positions()

#for _ in range(len(posits)): 
#    print(posits[_].avgCost)

# End of Types of Functions

In [ ]:
tester = ib.trades()
tester

In [ ]:
pending_stop_loss_orders = pd.DataFrame()

def order_status(trade):
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        for index, row in pending_stop_loss_orders.iterrows():
            if(row['Ticker'] == fill.contract.symbol and row['Quantity'] == fill.execution.shares):
                
                contract = Stock(fill.contract.symbol, 'SMART', 'USD')
                stopOrder = StopOrder('SELL', fill.execution.shares, row['SLTarget'])
                stopOrderTrade = ib.placeOrder(contract, stopOrder)
                
                pending_stop_loss_orders.drop(row)   # no longer useful to keep (does this work?)
                break
                
                
contract = Stock("SYRS", 'SMART', 'USD')
limitOrder = LimitOrder('BUY', 1, 0.19)
limitOrderTrade = ib.placeOrder(contract, limitOrder)
trade_details = limitOrderTrade

# CREATE A DATAFRAME THAT TRACKS STOP LIMIT ORDERS MADE ALONG WITH THEIR STOP LOSS TARGETS
new_row = pd.DataFrame({"Ticker": [trade_details.contract.symbol], "Quantity": [1], "SLTarget": [0.015]})
pending_stop_loss_orders = pd.concat([pending_stop_loss_orders, new_row], ignore_index=True)

limitOrderTrade.fillEvent += order_status

In [ ]:
tester = ib.executions()
tester

In [ ]:
tester = ib.fills()
tester

In [ ]:
tester = ib.openTrades()
tester

In [ ]:

contract = Stock("TSLA", 'SMART', 'USD')
limitStopOrder = StopLimitOrder('BUY', 1, 0.40, 0.50)
limitStopOrderTrade = ib.placeOrder(contract, limitStopOrder)
data = limitStopOrderTrade
print(data)

new_row = pd.DataFrame({"Ticker": [data.contract.symbol], "Quantity": [data.order.totalQuantity], "BuyInTarget": [data.order.lmtPrice], "StopLossTarget": [1.00]})
df = pd.concat([df, new_row], ignore_index=True)
print(df)
#bracket_LSO_Trade = ib.placeOrder(contract, bracket_LSO)
#bracket_LSO_Trade

#limitStopOrderTrade = ib.placeOrder(contract, limitStopOrder)

#bracketed_LSO = BracketOrder(limitStopOrderTrade, 0.60, 0.30)
                
#limitStopOrderTrade # execute   
#bracketed_LSO_Trade = ib.placeOrder(contract, bracketed_LSO) 
#bracketed_LSO_Trade # execute